In [2]:
import random

def f(x):
    return x**2 + 5

def df(x):
    return 2*x

old_x = float('inf')
x = random.randint(0,10000)
learning_rate = 0.3
epochs = 0

while abs(x-old_x) > 1.0e-7:
    cost = f(x)
    gradx = df(x)
    
    old_x = x
    x -= learning_rate * gradx
    
    print('EPOCJ{}: Cost={:.3f}, x={:.3f}'.format(epochs,cost,gradx))
    epochs += 1

EPOCJ0: Cost=27531014.000, x=10494.000
EPOCJ1: Cost=4404966.440, x=4197.600
EPOCJ2: Cost=704798.830, x=1679.040
EPOCJ3: Cost=112772.013, x=671.616
EPOCJ4: Cost=18047.722, x=268.646
EPOCJ5: Cost=2891.836, x=107.459
EPOCJ6: Cost=466.894, x=42.983
EPOCJ7: Cost=78.903, x=17.193
EPOCJ8: Cost=16.824, x=6.877
EPOCJ9: Cost=6.892, x=2.751
EPOCJ10: Cost=5.303, x=1.100
EPOCJ11: Cost=5.048, x=0.440
EPOCJ12: Cost=5.008, x=0.176
EPOCJ13: Cost=5.001, x=0.070
EPOCJ14: Cost=5.000, x=0.028
EPOCJ15: Cost=5.000, x=0.011
EPOCJ16: Cost=5.000, x=0.005
EPOCJ17: Cost=5.000, x=0.002
EPOCJ18: Cost=5.000, x=0.001
EPOCJ19: Cost=5.000, x=0.000
EPOCJ20: Cost=5.000, x=0.000
EPOCJ21: Cost=5.000, x=0.000
EPOCJ22: Cost=5.000, x=0.000
EPOCJ23: Cost=5.000, x=0.000
EPOCJ24: Cost=5.000, x=0.000
EPOCJ25: Cost=5.000, x=0.000
EPOCJ26: Cost=5.000, x=0.000
EPOCJ27: Cost=5.000, x=0.000


In [3]:
import numpy as np

class Node(object):
    def __init__(self,inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        #set 'self' node as inbound_noes's outbound_nodes 
        
        self.value = None
        
        self.gradients = {}
        #keys are the inputs to this node and their
        #values are the partials of this node with respect
        #to that input.
        #\partial{node}{input_i}
    
    def forward(self):
        '''
        Forward propagation.
        Compute the output values based on  'inbound_nodes' and
        store the result in self.value
        '''
        raise NotImplemented
        
    def backward(self):
        
        raise NotImplemented

In [28]:
class Input(Node):
    def __init__(self):
        '''
        An Input node has no inbound bodes.
        So no need to pass anything to the Node instanttiator.
        '''
        Node.__init__(self)
        
    def forward(self, value=None):
        '''
        Only the values in input node have to be passed to the forward().
        All the others should receive the value from the previous node 
        (self.inbound_nodes)
        
        Example:
        va10:self.inbound_node[0].value
        '''
        if value is not None:
            self.value = value
            
    def backward(self):
        self.gradients={self:0}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] = grad_cost * 1
        
        # input N --> N1,N2
        # \partial L / \partial N
        # ==> \partial L / \partial N1 * \partial N1 / \partial N

class CalculateNode(Node):
    def __init__(self,f,*nodes):
        Node.__init__(self,nodes)
        self.func = f
    
    def forward(self):
        self.value = self.func(map(lambda n: n.value, self.inbound_nodes))
        # this function will calculate the value of the current node , when forward is executed.

class Linear(Node):
    def __init__(self, nodes, weights, bias):
        Node.__init__(self,[nodes,weights,bias])
    
    def forward(self):
        inbound_nodes = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        
        self.value = np.dot(inbound_nodes,weights)+bias
    
    def backward(self):
        # initial a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        
        for n in self.outbound_nodes:
            # Get the partial of the cost w.r.t this node.
            grad_cost = n.gradients[self]
            
            self.gradients[self.inbound_nodes[0]] = np.dot(grad_cost, self.inbound_nodes[1].value.T)
            self.gradients[self.inbound_nodes[1]] = np.dot(self.inbound_nodes[0].value.T, grad_cost)
            self.gradients[self.inbound_nodes[2]] = np.sum(grad_cost, axis=0,keepdims=False)
            
            #WX+B / W ==> X
            #WX+B / X ==>W
            
class Sigmoid(Node):
    def __init__(self, node):
        Node.__init__(self, [node])
    
    def _sigmoid(self,x):
        return 1./(1+np.exp(-1*x))
    
    def forward(self):
        self.x = self.inbound_nodes[0].value
        self.value = self._sigmoid(self.x)
    
    def backward(self):
        self.partial = self._sigmoid(self.x)*(1-self._sigmoid(self.x))
        
        # y = 1 / (1+e^-x)
        # y'= 1 /(1+e^-x)(1-1/(1+e^-x))
        self.gradients = {n:np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self] #Get the partial of the cost with respect to this node
            
        self.gradients[self.inbound_nodes[0]] = grad_cost * self.partial
        # use * to keep all the dimension!
            
class MSE(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y,a])
    
    def forward(self):
        y = self.inbound_nodes[0].value.reshape(-1,1)
        a = self.inbound_nodes[1].value.reshape(-1,1)
        assert(y.shape == a.shape)
        
        self.m = self.inbound_nodes[0].value.shape[0]
        self.diff = y-a
        
        self.value = np.mean(self.diff**2)
    
    def backward(self):
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff
    
def forward_and_backward(outputnode, graph):
    # excute all the forward method of sorted_nodes.
    ## in practice, it is common to feed in mutiple data example in each forward pass rather than just 1.
    ## because the exampls can be processed in parallel. The number of examples is called batch size.
    for n in graph:
        n.forward()
       # each node execute forward, get self.value based on the topological sorted result.
        
    for n in graph[::-1]:
        n.backward()
        # return outputnode.value
## v --> a --> c
## b --> C
## b -->v -->a -->c
## v -->v --> a -->c
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm
    'feed_dict': A dictionary where the key is a 'Input' node and the value is the respective value feed to that node.
    Returns a list of sorted nodes.
    """
    input_nodes = [n for n in feed_dict.keys()]
    
    G={}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n  not in G:
            G[n] = {'in': set(), 'out':set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in':set(),'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
            
    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()
        
        if isinstance(n, Input):
            n.value = feed_dict[n]
            ## if n is Input Node, set n.value as
            ## feed_dict[n]
            ## else, n's value is caculate as its inbounds
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

def sgd_update(trainables, learning_rate=1e-2):
    for t in trainables:
        t.value -= learning_rate * t.gradients[t]

In [29]:
from sklearn.datasets import load_boston

In [30]:
data=load_boston()

In [31]:
"""
Check out the new network architecture and dataset!
Notice that the weights and biases are generated randomly.
No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size etc!
"""
import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

#Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

#Normalize data
X_ = (X_ - np.mean(X_,axis=0))/np.std(X_,axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden,1)
b2_ = np.zeros(1)

#Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 5000
# Total number of examples
m = X_.shape[0]
batch_size = 16
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        #Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples = batch_size)
        
        # Reset value of X and y Input
        X.value = X_batch
        y.value = y_batch
        
        # Step 2
        _ = None
        forward_and_backward(_,graph)
        
        # Step 3
        rate =1e-2
        
        sgd_update(trainables, rate)
        
        loss += graph[-1].value
        
        if i%100 == 0:
            print('Epoch: {}, Loss: {:.3f}'.format(i+1, loss/steps_per_epoch))

Total number of examples = 506
Epoch: 1, Loss: 21.382
Epoch: 1, Loss: 33.644
Epoch: 1, Loss: 44.811
Epoch: 1, Loss: 52.369
Epoch: 1, Loss: 61.135
Epoch: 1, Loss: 68.180
Epoch: 1, Loss: 76.966
Epoch: 1, Loss: 83.839
Epoch: 1, Loss: 86.731
Epoch: 1, Loss: 95.028
Epoch: 1, Loss: 98.956
Epoch: 1, Loss: 104.270
Epoch: 1, Loss: 108.802
Epoch: 1, Loss: 112.999
Epoch: 1, Loss: 117.951
Epoch: 1, Loss: 118.515
Epoch: 1, Loss: 120.010
Epoch: 1, Loss: 120.734
Epoch: 1, Loss: 121.157
Epoch: 1, Loss: 122.114
Epoch: 1, Loss: 124.255
Epoch: 1, Loss: 125.891
Epoch: 1, Loss: 127.570
Epoch: 1, Loss: 127.994
Epoch: 1, Loss: 128.508
Epoch: 1, Loss: 130.964
Epoch: 1, Loss: 131.763
Epoch: 1, Loss: 132.223
Epoch: 1, Loss: 134.215
Epoch: 1, Loss: 135.109
Epoch: 1, Loss: 135.946
Epoch: 101, Loss: 0.277
Epoch: 101, Loss: 0.401
Epoch: 101, Loss: 0.807
Epoch: 101, Loss: 1.464
Epoch: 101, Loss: 1.556
Epoch: 101, Loss: 1.986
Epoch: 101, Loss: 2.103
Epoch: 101, Loss: 2.380
Epoch: 101, Loss: 3.003
Epoch: 101, Loss: 3.

Epoch: 1101, Loss: 0.057
Epoch: 1101, Loss: 0.153
Epoch: 1101, Loss: 0.261
Epoch: 1101, Loss: 0.415
Epoch: 1101, Loss: 0.494
Epoch: 1101, Loss: 0.665
Epoch: 1101, Loss: 0.940
Epoch: 1101, Loss: 1.074
Epoch: 1101, Loss: 1.176
Epoch: 1101, Loss: 1.428
Epoch: 1101, Loss: 1.586
Epoch: 1101, Loss: 1.744
Epoch: 1101, Loss: 1.797
Epoch: 1101, Loss: 1.940
Epoch: 1101, Loss: 1.987
Epoch: 1101, Loss: 2.261
Epoch: 1101, Loss: 2.433
Epoch: 1101, Loss: 2.566
Epoch: 1101, Loss: 2.702
Epoch: 1101, Loss: 2.810
Epoch: 1101, Loss: 2.847
Epoch: 1101, Loss: 2.962
Epoch: 1101, Loss: 3.072
Epoch: 1101, Loss: 3.233
Epoch: 1101, Loss: 3.438
Epoch: 1101, Loss: 3.511
Epoch: 1101, Loss: 3.564
Epoch: 1101, Loss: 3.695
Epoch: 1101, Loss: 3.845
Epoch: 1101, Loss: 4.147
Epoch: 1101, Loss: 4.208
Epoch: 1201, Loss: 0.106
Epoch: 1201, Loss: 0.342
Epoch: 1201, Loss: 0.389
Epoch: 1201, Loss: 0.581
Epoch: 1201, Loss: 0.674
Epoch: 1201, Loss: 0.785
Epoch: 1201, Loss: 0.827
Epoch: 1201, Loss: 0.937
Epoch: 1201, Loss: 1.145


Epoch: 2201, Loss: 0.159
Epoch: 2201, Loss: 0.247
Epoch: 2201, Loss: 0.454
Epoch: 2201, Loss: 0.490
Epoch: 2201, Loss: 0.594
Epoch: 2201, Loss: 0.695
Epoch: 2201, Loss: 0.790
Epoch: 2201, Loss: 0.903
Epoch: 2201, Loss: 1.006
Epoch: 2201, Loss: 1.229
Epoch: 2201, Loss: 1.316
Epoch: 2201, Loss: 1.402
Epoch: 2201, Loss: 1.472
Epoch: 2201, Loss: 1.564
Epoch: 2201, Loss: 1.611
Epoch: 2201, Loss: 1.697
Epoch: 2201, Loss: 1.785
Epoch: 2201, Loss: 1.900
Epoch: 2201, Loss: 1.958
Epoch: 2201, Loss: 2.029
Epoch: 2201, Loss: 2.105
Epoch: 2201, Loss: 2.339
Epoch: 2201, Loss: 2.422
Epoch: 2201, Loss: 2.529
Epoch: 2201, Loss: 2.738
Epoch: 2201, Loss: 2.853
Epoch: 2201, Loss: 2.997
Epoch: 2201, Loss: 3.090
Epoch: 2201, Loss: 3.245
Epoch: 2201, Loss: 3.339
Epoch: 2201, Loss: 3.464
Epoch: 2301, Loss: 0.075
Epoch: 2301, Loss: 0.217
Epoch: 2301, Loss: 0.371
Epoch: 2301, Loss: 0.440
Epoch: 2301, Loss: 0.569
Epoch: 2301, Loss: 0.678
Epoch: 2301, Loss: 0.731
Epoch: 2301, Loss: 0.956
Epoch: 2301, Loss: 1.114


Epoch: 3301, Loss: 0.095
Epoch: 3301, Loss: 0.337
Epoch: 3301, Loss: 0.501
Epoch: 3301, Loss: 0.843
Epoch: 3301, Loss: 1.039
Epoch: 3301, Loss: 1.150
Epoch: 3301, Loss: 1.403
Epoch: 3301, Loss: 1.491
Epoch: 3301, Loss: 1.565
Epoch: 3301, Loss: 1.680
Epoch: 3301, Loss: 1.820
Epoch: 3301, Loss: 1.998
Epoch: 3301, Loss: 2.117
Epoch: 3301, Loss: 2.171
Epoch: 3301, Loss: 2.319
Epoch: 3301, Loss: 2.393
Epoch: 3301, Loss: 2.579
Epoch: 3301, Loss: 2.760
Epoch: 3301, Loss: 2.907
Epoch: 3301, Loss: 2.966
Epoch: 3301, Loss: 3.087
Epoch: 3301, Loss: 3.181
Epoch: 3301, Loss: 3.356
Epoch: 3301, Loss: 3.469
Epoch: 3301, Loss: 3.603
Epoch: 3301, Loss: 3.719
Epoch: 3301, Loss: 3.902
Epoch: 3301, Loss: 3.987
Epoch: 3301, Loss: 4.088
Epoch: 3301, Loss: 4.210
Epoch: 3301, Loss: 4.437
Epoch: 3401, Loss: 0.065
Epoch: 3401, Loss: 0.146
Epoch: 3401, Loss: 0.242
Epoch: 3401, Loss: 0.381
Epoch: 3401, Loss: 0.540
Epoch: 3401, Loss: 0.617
Epoch: 3401, Loss: 0.714
Epoch: 3401, Loss: 0.872
Epoch: 3401, Loss: 1.014


Epoch: 4401, Loss: 0.087
Epoch: 4401, Loss: 0.186
Epoch: 4401, Loss: 0.243
Epoch: 4401, Loss: 0.334
Epoch: 4401, Loss: 0.415
Epoch: 4401, Loss: 0.490
Epoch: 4401, Loss: 0.559
Epoch: 4401, Loss: 0.709
Epoch: 4401, Loss: 0.959
Epoch: 4401, Loss: 1.062
Epoch: 4401, Loss: 1.152
Epoch: 4401, Loss: 1.238
Epoch: 4401, Loss: 1.298
Epoch: 4401, Loss: 1.443
Epoch: 4401, Loss: 1.567
Epoch: 4401, Loss: 1.683
Epoch: 4401, Loss: 1.783
Epoch: 4401, Loss: 1.940
Epoch: 4401, Loss: 2.185
Epoch: 4401, Loss: 2.278
Epoch: 4401, Loss: 2.622
Epoch: 4401, Loss: 2.769
Epoch: 4401, Loss: 2.941
Epoch: 4401, Loss: 3.118
Epoch: 4401, Loss: 3.320
Epoch: 4401, Loss: 3.398
Epoch: 4401, Loss: 3.445
Epoch: 4401, Loss: 3.605
Epoch: 4401, Loss: 3.840
Epoch: 4401, Loss: 3.984
Epoch: 4401, Loss: 4.049
Epoch: 4501, Loss: 0.069
Epoch: 4501, Loss: 0.135
Epoch: 4501, Loss: 0.227
Epoch: 4501, Loss: 0.345
Epoch: 4501, Loss: 0.441
Epoch: 4501, Loss: 0.501
Epoch: 4501, Loss: 0.669
Epoch: 4501, Loss: 0.771
Epoch: 4501, Loss: 0.975
